In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import time
import hashlib, hmac, base64
import requests, json, urllib3

requests.packages.urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import sys

SPINNING_RATE = 0.1
TIME_ADJUST = 0
PRINT_TRACE = 0

# config is passed to most function, this is a json object that includes all the required parameters

def spinning_cursor():
    while True:
        for cursor in '|/-\\':
            yield cursor

def init_params_headers(config, headers, params):
    params.update({'org' : config['org_key']})     
    headers.update({"content-type": "application/json", "Authorization": "Bearer %s" % config['sign'] })

def ws_post_sign(config):

    url = "%s/integration/sign" % config['url']
    headers = {"content-type": "application/json"}
    data = { 'uid' : config['user_id'], 'apiKey' : config['api_key']}
    if( PRINT_TRACE) : print("REST CALL: "+url+" "+str(data))
    r = requests.post(url, verify=False, data=json.dumps(data), headers=headers)
    if (r.status_code != 200):
        print("Error: get signature: %s" % r.json()['data'])
        config['sign'] = 0
        return 0
    else:
        config['sign'] = r.json()['sign']
    return r.json()['sign']


def ws_delete_process(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s" % (config['url'], config['project_key'])

    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.delete(url, verify=False, params=params, headers=headers)
    if (r.status_code != 200):
        print("Error: delete process: %s" % r.json()['data'])        
    return r

def ws_proc_post(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes" % (config['url'])
    data = { 'title' : config['project_name'], 'org' : config['org_key']}
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(data))
    r = requests.post(url, verify=False, data=json.dumps(data), headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: Process %s creation: %s" % (config['project_name'], r.json()['data']))        
    return r

def ws_csv_upload(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    # no content-type 
    headers.pop("content-type")

    url = "%s/integration/csv/%s/upload" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    files = {'file': (config['csv_filename'], open(config['csv_filename'], 'rb'),'text/zip')}
    r = requests.post(url, verify=False, files=files, params=params, headers=headers)
    # Async call --- the caller need to loop on get job status that it is completed
    if (r.status_code != 200):
        print("Error: CSV upload: %s" % r.json()['data'])        
    return r


def ws_query_post(config, query):
    headers = {}
    params = {}
    init_params_headers(config, headers, params)
    headers['content-type'] = 'application/x-www-form-urlencoded'
    url = "%s/analytics/integration/%s/query" % (
        config['url'], config['project_key'])
    data = "params={'query': '%s'}" % query
    print(data)
    r = requests.post(url, verify=False, params=params,
                      headers=headers, data=data)
    if (r.status_code != 200):
        print("Error: query post: %s" % r.json()['data'])
    return r


def ws_backup_upload(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    # no content-type 
    headers.pop("content-type")

    url = "%s/integration/processes/%s/upload-backup" % (config['url'], config['project_key'])

    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    files = {'file': (config['backup_filename'], open(config['backup_filename'], 'rb'),'text/zip')}
    r = requests.post(url, verify=False, files=files, params=params, headers=headers)
    if (r.status_code != 200):
        print("Error: backup upload: %s" % r.json()['data']) 
    return r

def ws_get_backup_list(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url  = "%s/integration/processes/%s/backups" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.get(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: backup list get: %s" % r.json()['data'])
    return r

def getBackupIdByMessage(backuplist, message):
    backuplist = backuplist['backups']
    for backup in backuplist:
        if (backup['message'] == message) :
            return backup['id']
    return 0

def ws_apply_backup(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/backups/%s" % (config['url'], config['project_key'], config['backup_id'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.put(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: apply backup: %s : %s" % (r.json()['data'], config['backup_id'])) 
    return r

def ws_create_log(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/%s/create-log" % (config['url'], config['project_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.post(url, verify=False, headers=headers, params=params)
    # Async function, the caller needs to check the job status (returned in r)
    if (r.status_code != 200):
        print("Error: create log: %s" % r.json()['data']) 
    return r

def ws_get_csv_job_status(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/job-status/%s" % (config['url'], config['job_key'])
    if( PRINT_TRACE) : print("REST CALL: "+ url+" "+str(params))
    r = requests.get(url, verify=False, headers=headers, params=params)
    if (r.status_code != 200):
        print("Error: get CSV job status: %s" % r.json()['data'])        
    return r

def create_and_load_new_project(config):
    print("Process Mining: creating new project")
    r_proc = ws_proc_post(config)
    if (r_proc.status_code != 200): return r_proc
    config['project_key']= r_proc.json()['projectKey']
    print("Process Mining: loading event log (please wait)")
    r = ws_csv_upload(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while loading CSV -- column number mismatch")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)

    r = ws_backup_upload(config)
    if (r.status_code != 200): return r
    config['backup_id'] = r.json()['backupInfo']['id']
    r = ws_apply_backup(config)
    if (r.status_code != 200): return r
    print("Process Mining: refreshing process (please wait)")
    r = ws_create_log(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        #pbar = tqdm(total=100)
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while creating the log")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)
    return r_proc

def upload_csv_and_createlog(config) :
    r = ws_csv_upload(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        print("Process Mining: loading event log (please wait)")
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while loading CSV -- column number mismatch")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)

    r = ws_create_log(config)
    if (r.status_code != 200): return r
    else :
        # wait until async call is completed
        config['job_key']= r.json()['data']
        runningCall = 1
        print("Process Mining: refreshing model (please wait)")
        spinner = spinning_cursor()
        while  runningCall :
            r = ws_get_csv_job_status(config)
            if (r.json()['data'] == 'complete') :
                runningCall = 0
            if (r.json()['data'] == 'error') :
                runningCall = 0
                print("Error while creating the log")
                return 0
            sys.stdout.write(next(spinner))
            sys.stdout.flush()
            sys.stdout.write('\b')
            time.sleep(SPINNING_RATE)
    return r

# List all the dashboards of a given process mining project
def ws_get_dashboards(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/list" % (config['url'], config['project_key'])
    r = requests.get(url, verify=False, params=params, headers=headers)
    if (r.status_code == 200):
        values = r.json()
        return values['data']['dashboards']
    else:
        return r

# Returns only the table widgets in the dashboard_name
def ws_get_widgets(config):
    dashboards = ws_get_dashboards(config)
    dashboard = 0
    for aDashboard in dashboards:
        if (aDashboard['name'] == config['dashboard_name']):
            dashboard = aDashboard

    if dashboard == 0 :
        print("ERROR: dashboard does not exist")
        return 0
    dashboard_id = dashboard['id']

    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/%s/list" % (config['url'], config['project_key'], dashboard_id)

    r = requests.get(url, verify=False, params=params, headers=headers )
    widgets = r.json()
    return widgets['data']['widgets']

def ws_get_widget_values(config):
    dashboard_id = config.get('dashboard_id')
    if dashboard_id is None:
        # add the dashboard_id to the widget data
        print("....searching dashboard_id")
        dashboards = ws_get_dashboards(config)
        dashboard = 0
        for aDashboard in dashboards:
            if (aDashboard['name'] == config['dashboard_name']):
                dashboard = aDashboard
        if (dashboard == 0) :
            print("ERROR: dashboard does not exist")
            return 0
        else:
            # Store the dashboard id in the widget to avoid calling again the rest API to retrieve the dashboard by name
            config['dashboard_id'] = dashboard['id']

    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/analytics/integration/dashboard/%s/%s/%s/retrieve" % (config['url'],
                                                                        config['project_key'],
                                                                        config['dashboard_id'],
                                                                        config['widget_id'])

    r = requests.get(url, verify=False, params=params, headers=headers )
    values = r.json()
    return values['data']

def ws_create_update_variables(config, variablesArray):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables" % (config['url'], config['project_key'])
    r = requests.post(url, verify=False, data=json.dumps(variablesArray), params=params, headers=headers )
    return r

def ws_get_variable(config, variablename):
    headers={}
    params={}
    init_params_headers(config, headers, params)
    
    url = "%s/integration/processes/%s/variables/%s" % (config['url'], config['project_key'], variablename)
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return 0

def ws_get_variables(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables" % (config['url'], config['project_key'])
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        print("Error: ws_get_variables %s" % r.json()['data'])
        return []

def ws_delete_variable(config, variablename):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s/variables/%s" % (config['url'], config['project_key'], variablename)
    r = requests.delete(url, verify=False, params=params, headers=headers )
    return r.status_code

def ws_delete_variables(config):
    variables = ws_get_variables(config)
    for variable in variables:
        ws_delete_variable(config, variable['name'])

def ws_get_processes(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes" % (config['url'])
    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return r

def ws_get_project_info(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/processes/%s" % (config['url'], config['project_key'])

    return(requests.get(url, verify=False, params=params, headers=headers ))


def ws_get_project_meta_info(config):
    headers={}
    params={}
    init_params_headers(config, headers, params)

    url = "%s/integration/csv/%s/meta" % (config['url'], config['project_key'])

    r = requests.get(url, verify=False, params=params, headers=headers )
    if (r.status_code == 200):
        values = r.json()
        return values['data']
    else:
        return r



In [18]:
import random
import json
import time
from datetime import datetime as dt, timedelta
import sys
import pandas as pd

def load_history_df(history_file):
    try: 
        histo_df = pd.read_csv(history_file, dtype=str)
        print('Loading history file: %s' % history_file)
        return histo_df
    except:
        # First call: Create the historical CSV from scratch at the end
        print('First time execution, no history df')
        return pd.DataFrame()

def manageAlertsFromWidget(myconfig):

    # Example of a configuration object that could be stored as a config file passed as
    # a parameter of the program
    # custom_data are added to the alert fetched from the widget. They can be used to manage
    # some business logic with the alerts received
    #if (len(argv)==2):
    #    try:
    #        with open(argv[1], 'r') as file:
    #            myconfig = json.load(file)
    #            print('Loading config file')
    #           print(myconfig)
    #    except:
    #        print('Error config file not found')
    #else:
    #    myconfig = {
    #        "url":"https://ProcessMining.com",
    #        "user_id": "john.smith",
    #        "api_key":"8a5kga87eqvd1180",
    #        "project_key": "procure-to-pay",
    #        "org_key": "",
    #        "dashboard_name": "Alerts Dashboard",
    #        "widget_id": "alerts-widget-1",
    #        "custom_data": {"Status":"False", "Custom2":"default"}
    #    }

        
    ws_post_sign(myconfig)

    # Get the current values from the widget
    widgetAlerts = ws_get_widget_values(myconfig)
    print("%d current alerts for widget %s in dashboard %s" % (len(widgetAlerts), myconfig['widget_id'], myconfig['dashboard_name']))     
    widget_df = pd.DataFrame(widgetAlerts)

    # if the alert_history file does not exist, create it
    history_file = 'alert_history_' + myconfig['dashboard_name'] + '_' + myconfig['widget_id']+'.csv'
    summary_file = 'alert_summary_' + myconfig['dashboard_name'] + '_' + myconfig['widget_id']+'.csv'

    # Load alert history for this widget
    histo_df = load_history_df(history_file)
    if (len(histo_df) == 0):
        # First time execution
        if (len(widget_df) ==  0): # No alerts in the widget
            print('Empty widget, nothing to do')
            return
        else: # create the first history file and summary file and quit
            histo_df = pd.DataFrame(widgetAlerts)
            histo_df['alert_status'] = 'NEW'
            histo_df['alert_creation_date'] = dt.now().isoformat()
            histo_df['alert_closed_date'] = ''
            histo_df.to_csv(history_file, index=None)
            summary = {
                'update_date': dt.now().isoformat(),
                #'last_update_date': 0,
                #'last_new': 0,
                #'last_pending': 0,
                #'last_closed': 0,
                'new': len(histo_df),
                'pending': 0,
                'closed': 0,
                'new_to_pending': 0,
                'new_to_closed': 0,
                'pending_to_closed': 0,
                'pending_to_pending': 0,
                'any_to_closed': 0,
                'progression_rate': 0
            }
            summary_df = pd.DataFrame([summary])
            summary_df.to_csv(summary_file, index=None)
            return

    # Load summary history for this widget
    try: 
        summary_df = pd.read_csv(summary_file)
        print('Loading summary file: %s' % summary_file)
        last_summary = summary_df.loc[len(summary_df) - 1]
    except:
        # First call: Create the summary df 
        summary_df = pd.DataFrame()
        last_summary = {
            'update_date': 0,
            'new': 0,
            'pending': 0,
            'closed': 0,
            'new_to_pending': 0,
            'new_to_closed': 0,
            'pending_to_closed': 0,
            'pending_to_pending': 0,
            'any_to_closed': 0,
            'progression_rate': 0
        }
    summary = {
        'update_date': dt.now().isoformat(),
        'new': 0,
        'pending': 0,
        'closed': last_summary['closed'],
        'new_to_pending': 0,
        'new_to_closed': 0,
        'pending_to_closed': 0,
        'pending_to_pending': 0,
        'any_to_closed': 0,
        'progression_rate': 0
    }

    # Collect the number of alerts in history for each status
    histo_closed_df = histo_df[histo_df['alert_status'] ==  'CLOSED']
    final_df = histo_closed_df # we keep the closed alerts

    # History: CLOSED alerts in histo are still closed
    if (last_summary['closed']):
        print('%d alerts already CLOSED' % last_summary['closed'])

    # If widget is empty, all the alerts are CLOSED
    if (len(widget_df) == 0):
        print("Empty widget, all alerts are closed")
        summary['new'] = 0
        summary['pending'] = 0
        summary['closed'] = len(histo_df)
        summary['new_to_closed'] = last_summary['new']
        summary['pending_to_closed'] = last_summary['pending']
        summary['any_to_closed'] = summary['new_to_closed'] + summary['pending_to_closed']
        summary['pending_to_pending'] = 0

        histo_any_to_close_df = histo_df[histo_df['alert_status'] != 'CLOSED']
        histo_any_to_close_df['alert_closed_date'] = dt.now().isoformat()
        histo_any_to_close_df['alert_status'] =  'CLOSED'
        final_df = pd.concat([final_df, histo_any_to_close_df])
        final_df.to_csv(history_file, index=None)


    else: # Widget is not empty
        
        # We remove CLOSED alerts from histo_df, such that we can add same alerts again if they appear in the widget
        histo_not_closed_df = histo_df[histo_df['alert_status'] != 'CLOSED']

        # NEW alerts in the widget
        widget_new_df = widget_df.merge(histo_not_closed_df, on=widget_df.columns.tolist(), how='left', indicator='alert_is_present_on')
        # Alerts in the widget with 'exist'==left_only are NEW
        widget_new_df = widget_new_df[widget_new_df['alert_is_present_on'] == 'left_only']
        summary['new'] = len(widget_new_df)
        if (summary['new']):
            widget_new_df['alert_status'] = 'NEW'
            widget_new_df['alert_creation_date'] = dt.now().isoformat()
            widget_new_df['alert_closed_date'] = ''
            print('%d new alerts' % summary['new'])
            final_df = pd.concat([final_df, widget_new_df.drop(columns=['alert_is_present_on'])])


        # Process alerts that are in the HISTORY
        histo_not_closed_df = histo_not_closed_df.merge(widget_df, on=widget_df.columns.tolist(), how='left', indicator='alert_is_present_on')
        # NEW to PENDING  
        histo_new_to_pending_df = histo_not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="both"')
        summary['new_to_pending'] = len(histo_new_to_pending_df)
        summary['pending'] =  summary['new_to_pending']                                
        if (summary['new_to_pending']):
            print('%d alerts moved from NEW to PENDING' % summary['new_to_pending'])
            histo_new_to_pending_df['alert_status'] = 'PENDING'
            final_df = pd.concat([final_df, histo_new_to_pending_df.drop(columns=['alert_is_present_on'])])

        # NEW to CLOSE
        histo_new_to_close_df = histo_not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="left_only"')
        summary['new_to_closed'] = len(histo_new_to_close_df)
        summary['closed'] += summary['new_to_closed']                              
        if (summary['new_to_closed']):
            print('%d alerts moved from NEW to CLOSED' % summary['new_to_closed'])
            histo_new_to_close_df['alert_status'] = 'CLOSED'
            histo_new_to_close_df['alert_closed_date'] = dt.now().isoformat()
            final_df = pd.concat([final_df, histo_new_to_close_df.drop(columns=['alert_is_present_on'])]) 

        # PENDING to PENDING
        histo_pending_to_pending_df = histo_not_closed_df.query('alert_status=="PENDING" & alert_is_present_on=="both"')
        summary['pending_to_pending'] = len(histo_pending_to_pending_df)
        summary['pending'] += summary['pending_to_pending']
        if (summary['pending_to_pending']):
            print('%d alerts still PENDING' % summary['pending_to_pending'])
            final_df = pd.concat([final_df, histo_pending_to_pending_df.drop(columns=['alert_is_present_on'])])

        # PENDING to CLOSE
        histo_pending_to_close_df = histo_not_closed_df.query('alert_status=="PENDING" & alert_is_present_on=="left_only"')
        summary['pending_to_closed'] = len(histo_pending_to_close_df)
        summary['closed'] += summary['pending_to_closed']                              

        if (summary['pending_to_closed']):
            print('%d alerts moved from PENDING to CLOSE' % summary['pending_to_closed'])
            final_df = pd.concat([final_df, histo_pending_to_close_df.drop(columns=['alert_is_present_on'])]) 

    # Save final_df in the history file
    final_df.to_csv(history_file, index=None)

    # Add the summary object to the summary dataframe
    if (last_summary['pending'] + last_summary['new']):
        summary['progression_rate'] = summary['any_to_closed'] / (last_summary['pending'] + last_summary['new'])
    else:
        summary['progression_rate'] = 0.0
    print('Progression rate: %d' % summary['progression_rate'])
    summary_df.loc[len(summary_df)] = summary
    summary_df.to_csv(summary_file, index=None)

config = {
    "url":"https://pharoses1.fyre.ibm.com",
    "user_id": "task.miner",
    "api_key":"8a5kga87eqvd1180",
    "project_key": "procure-to-pay",
    "org_key": "",
    "dashboard_name": "Alerts Dashboard",
    "widget_id": "alerts-widget-1",
    "custom_data": {"Status":"False", "Custom2":"default"}
}


manageAlertsFromWidget(config)

....searching dashboard_id
3 current alerts for widget alerts-widget-1 in dashboard Alerts Dashboard
Loading history file: alert_history_Alerts Dashboard_alerts-widget-1.csv
Loading summary file: alert_summary_Alerts Dashboard_alerts-widget-1.csv
3 alerts already CLOSED
3 alerts still PENDING
Progression rate: 0
